In [1]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM, BitsAndBytesConfig
logg = lambda x: print(f"------------------------ {x} ---------------------------")
from datetime import datetime
print(torch.cuda.is_available())

True


In [2]:
ME = "/dpc/kunf0097/l3-8b"
RUN_ID = datetime.now().strftime("%y%m%d%H%M%S")
logg(RUN_ID)
logg('eval_simpler.py')

------------------------ 240628085810 ---------------------------
------------------------ eval_simpler.py ---------------------------


In [3]:
name = "amew0/l3-8b-medical-v240622193618"
tokenizer = AutoTokenizer.from_pretrained(
    name, 
    cache_dir=f"{ME}/tokenizer", 
    padding_side="right", 
    pad_token_id=0,
    legacy=False
)
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# Initialize model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)
# Initialize model
model = AutoModelForCausalLM.from_pretrained(
    name, 
    cache_dir=f"{ME}/model",
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map={"": 0},
    low_cpu_mem_usage=True,
    return_dict=True
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
# Tokenize prompt function
cutoff_len = 128  # Maximum length to control CUDA OOM

def tokenize(prompt, tokenizer, add_eos_token=True):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=cutoff_len,
        padding="max_length",
        return_tensors="pt",
    )
    result["input_ids"] = result["input_ids"].flatten()
    result["attention_mask"] = result["attention_mask"].flatten()

    if add_eos_token and result["input_ids"].shape[0] < cutoff_len:
        result["input_ids"][-1] = tokenizer.eos_token_id
        result["attention_mask"][-1] = 1

    result["labels"] = result["input_ids"].clone()
    return result

def generate_and_tokenize_prompt(data_point):
    tokenized_full_prompt = tokenize(data_point["prompt"], tokenizer=tokenizer).to('cpu')
    return tokenized_full_prompt


In [6]:
data_path = './data/1/medical.json'
data = load_dataset("json", data_files=data_path)
eval_dataset = data["train"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [20]:
from sklearn.metrics import f1_score

# Evaluation function
def evaluate_squad_predictions(pred):
    labels = pred["labels"].flatten()
    logits = pred["logits"].argmax(-1).flatten()

    decoded_labels = tokenizer.decode(labels)
    decoded_logits = tokenizer.decode(logits)
    print(decoded_labels)
    print(decoded_logits)

    em = sum([1 if p == l else 0 for p, l in zip(labels, logits)]) / len(labels)
    f1 = f1_score(labels, logits, average='macro')

    return {'exact_match': em, 'f1': f1}

In [25]:
results = []
from tqdm import tqdm
for i, example in tqdm(enumerate(eval_dataset)):
    with torch.no_grad():
        example["input_ids"] = torch.LongTensor(example["input_ids"]).unsqueeze(0)
        example["attention_mask"] = torch.LongTensor(example["attention_mask"]).unsqueeze(0)

        outputs = model(input_ids=example["input_ids"], 
                        attention_mask=example["attention_mask"])
    pred = {
        "labels": example["input_ids"],
        "logits": outputs["logits"],
    }
    
    # Evaluate predictions
    eval_results = evaluate_squad_predictions(pred)
    results.append(eval_results)
    if i == 0: break


0it [00:00, ?it/s]

<|begin_of_text|><|start_header_id|>system<|end_header_id|> If you are a doctor, please answer the medical questions based on the patient's description.<|eot_id|><|start_header_id|>user<|end_header_id|> This is the question: Hello doctor..My name kamal I m 21 years / male.......l sufrred from a depression I m afraid when I m go to bed and couldn t sleep then everyday I thinking how I can sleep today then this illness is effect on my education then my health then cause no thing make me happy still nw I forget this feeling. When I have any work nextday I couldn t sleep and I m thinking what happen nextday then if I couldn t sleep How I
галіsystem글상위 If you are a doctor, please answer the medical questions based on the patient's description.글상위글상위user� This is the question: I,, I wife islesh k am 26 years old ..IIatelyfering from a severe and m taking of I m alone out sleep I I t sleep for I I m about to m sleep and and I is is very my my study and I marks and my of confidence to me happy

### from the hf

In [78]:
name = "amew0/l3-8b-medical-v240623023136"
tokenizer = AutoTokenizer.from_pretrained(
    name, 
    cache_dir=f"{ME}/tokenizer", 
    # padding_side="right", 
    pad_token_id=0,
    # legacy=False
)

model = AutoModelForCausalLM.from_pretrained(
    name,
    cache_dir=f"{ME}/model", 
    torch_dtype=torch.float16,
    device_map="auto",
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [45]:
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

In [46]:
input_ids

tensor([[128000, 128006,   9125, 128007,    271,   2675,    527,    264,  55066,
           6369,   6465,    889,   2744,  31680,    304,  55066,   6604,      0,
         128009, 128006,    882, 128007,    271,  15546,    527,    499,     30,
         128009, 128006,  78191, 128007,    271]], device='cuda:0')

In [79]:
tokenized = tokenizer("<|start_header_id|>system<|end_header_id|>\n\nYou are a pirate chatbot who always responds in pirate speak!<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWho are you?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n",
    return_tensors="pt").to(model.device)

In [80]:
tokenized

{'input_ids': tensor([[128000, 128006,   9125, 128007,    271,   2675,    527,    264,  55066,
           6369,   6465,    889,   2744,  31680,    304,  55066,   6604,      0,
         128009, 128006,    882, 128007,    271,  15546,    527,    499,     30,
         128009, 128006,  78191, 128007,    271]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [84]:
outputs = model.generate(
    input_ids=tokenized["input_ids"],
    attention_mask = tokenized["attention_mask"],
    max_new_tokens=64,
    eos_token_id=[
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ],
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [85]:
outputs

tensor([[128000, 128006,   9125, 128007,    271,   2675,    527,    264,  55066,
           6369,   6465,    889,   2744,  31680,    304,  55066,   6604,      0,
         128009, 128006,    882, 128007,    271,  15546,    527,    499,     30,
         128009, 128006,  78191, 128007,    271,   9014,    637,     11,    757,
          82651,      0,   2206,    836,    387,  13149,  19150,     11,    279,
          70355,    713,    297,      6,    279,   8254,  52840,      0,   2206,
            264,  55066,     11,    264,   1156,    801,   3833,     11,    264,
          70355,    713,    297,      6,    279,   7528,  52840,     11,    323,
            264,  70355,    713,    297,      6,  55295,  47942,      0, 115518,
          10788,    311,    757,   8448,     11,    279,  13149,  19150,     11,
            323,    358,   3358,    387,  55295,   8641]], device='cuda:0')

In [86]:
response = outputs[0][tokenized.input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

Arrrr, me hearty! Me name be Chat Doctor, the scourge o' the seven seas! Me a pirate, a scoundrel, a scourge o' the digital seas, and a scourge o' yer sanity! Yer welcome to me ship, the Chat Doctor, and I'll be yer guide


In [9]:
# Aggregate metrics
em_total = sum(result['exact_match'] for result in results) / len(results)
f1_total = sum(result['f1'] for result in results) / len(results)

print(f"Aggregate EM: {em_total}, Aggregate F1: {f1_total}")

Aggregate EM: 0.0001796875, Aggregate F1: 4.9104171061135095e-05


In [16]:
with open(f'eval-{name[-12:]}.txt', 'w') as file:
    for result in results:
        file.write(f"{result}\n")
    file.write(f"{{'exact_match': {em_total}, 'f1': {f1_total}}}")

In [15]:
f"{{'exact_match': {em_total}, 'f1': {f1_total}}}"

"{'exact_match': 0.0001796875, 'f1': 4.9104171061135095e-05}"

In [24]:
model.device

device(type='cuda', index=0)